In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-addon-obp qiskit-addon-utils rustworkx

*Estimación de uso: 16 minutos en un procesador Eagle r3 (NOTA: Esta es solo una estimación. Su tiempo de ejecución podría variar.)*

In [ ]:
# This cell is hidden from users;
# it disables linting rules.
# ruff: noqa

## Antecedentes

La retropropagación de operadores es una técnica que consiste en absorber operaciones desde el final de un circuito cuántico en el observable medido, reduciendo generalmente la profundidad del circuito a costa de términos adicionales en el observable. El objetivo es retropropagar la mayor parte posible del circuito sin permitir que el observable crezca demasiado. Una implementación basada en Qiskit está disponible en el complemento OBP de Qiskit; se pueden encontrar más detalles en la [documentación](/guides/qiskit-addons-obp) correspondiente con un [ejemplo sencillo](/guides/qiskit-addons-obp-get-started) para comenzar.

Considera un circuito de ejemplo para el cual se debe medir un observable $O = \sum_P c_P P$, donde $P$ son operadores de Pauli y $c_P$ son coeficientes. Denotemos el circuito como un unitario único $U$ que puede particionarse lógicamente en $U = U_C U_Q$ como se muestra en la figura a continuación.

![Circuit diagram showing Uq followed by Uc](../docs/images/tutorials/improving-estimation-of-expectation-values-with-operator-backpropagation/logical-partitioning.avif)

La retropropagación de operadores absorbe el unitario $U_C$ en el observable al evolucionarlo como $O' = U_C^{\dagger}OU_C = \sum_P c_P U_C^{\dagger}PU_C$. En otras palabras, parte del cálculo se realiza clásicamente mediante la evolución del observable de $O$ a $O'$. El problema original ahora puede reformularse como la medición del observable $O'$ para el nuevo circuito de menor profundidad cuyo unitario es $U_Q$.

El unitario $U_C$ se representa como una cantidad de segmentos $U_C = U_S U_{S-1}...U_2U_1$. Existen múltiples formas de definir un segmento. Por ejemplo, en el circuito de ejemplo anterior, cada capa de puertas $R_{zz}$ y cada capa de puertas $R_x$ puede considerarse como un segmento individual. La retropropagación implica el cálculo de $O' = \Pi_{s=1}^S \sum_P c_P U_s^{\dagger} P U_s$ de manera clásica. Cada segmento $U_s$ puede representarse como $U_s = exp(\frac{-i\theta_s P_s}{2})$, donde $P_s$ es un operador de Pauli de $n$-qubits y $\theta_s$ es un escalar. Es fácil verificar que

$$
U_s^{\dagger} P U_s = P \qquad \text{if} ~[P,P_s] = 0,
$$
$$
U_s^{\dagger} P U_s = \qquad cos(\theta_s)P + i sin(\theta_s)P_sP \qquad \text{if} ~{P,P_s} = 0
$$

En el ejemplo anterior, si ${P,P_s} = 0$, entonces necesitamos ejecutar dos circuitos cuánticos, en lugar de uno, para calcular el valor esperado. Por lo tanto, la retropropagación puede aumentar el número de términos en el observable, lo que conduce a un mayor número de ejecuciones de circuitos. Una forma de permitir una retropropagación más profunda en el circuito, al tiempo que se evita que el operador crezca demasiado, es truncar los términos con coeficientes pequeños, en lugar de agregarlos al operador. Por ejemplo, en el caso anterior, se puede optar por truncar el término que involucra $P_sP$ siempre que $\theta_s$ sea suficientemente pequeño. Truncar términos puede resultar en menos circuitos cuánticos por ejecutar, pero hacerlo introduce cierto error en el cálculo final del valor esperado, proporcional a la magnitud de los coeficientes de los términos truncados.

Este tutorial implementa un [patrón de Qiskit](/guides/intro-to-patterns) para simular la dinámica cuántica de una cadena de espines de Heisenberg utilizando <a href="https://github.com/Qiskit/qiskit-addon-obp">qiskit-addon-obp</a>.

## Requisitos

Antes de comenzar este tutorial, asegúrate de tener instalado lo siguiente:

- Qiskit SDK v1.2 o posterior (`pip install qiskit`)
- Qiskit Runtime v0.28 o posterior (`pip install qiskit-ibm-runtime`)
- Complemento OBP de Qiskit (`pip install qiskit-addon-obp`)
- Utilidades del complemento de Qiskit (`pip install qiskit-addon-utils`)

## Configuración

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import CouplingMap
from qiskit.synthesis import LieTrotter

from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit_addon_utils.slicing import slice_by_gate_types, combine_slices
from qiskit_addon_obp.utils.simplify import OperatorBudget
from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.truncating import setup_budget

from rustworkx.visualization import graphviz_draw

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions

## Parte I: Cadena de espines de Heisenberg a pequeña escala
### Paso 1: Mapear las entradas clásicas a un problema cuántico
#### Mapear la evolución temporal de un modelo cuántico de Heisenberg a un experimento cuántico.
El paquete [qiskit_addon_utils](https://github.com/Qiskit/qiskit-addon-utils) proporciona algunas funcionalidades reutilizables para diversos propósitos.

Su módulo [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) ofrece funciones para generar hamiltonianos de tipo Heisenberg en un grafo de conectividad dado.
Este grafo puede ser un [rustworkx.PyGraph](https://www.rustworkx.org/apiref/rustworkx.PyGraph.html) o un [CouplingMap](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.CouplingMap), lo que facilita su uso en flujos de trabajo centrados en Qiskit.

A continuación, generamos un `CouplingMap` de cadena lineal de 10 qubits.

In [3]:
num_qubits = 10
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif)

A continuación, generamos un operador de Pauli que modela un hamiltoniano XYZ de Heisenberg.

$$
{\hat{\mathcal{H}}_{XYZ} = \sum_{(j,k)\in E} (J_{x} \sigma_j^{x} \sigma_{k}^{x} + J_{y} \sigma_j^{y} \sigma_{k}^{y} + J_{z} \sigma_j^{z} \sigma_{k}^{z}) + \sum_{j\in V} (h_{x} \sigma_j^{x} + h_{y} \sigma_j^{y} + h_{z} \sigma_j^{z})}
$$

Donde $G(V,E)$ es el grafo del mapa de acoplamiento proporcionado.

In [4]:
# Get a qubit operator describing the Heisenberg XYZ model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII', 'IIIIIIIIIX', 'IIIIIIIIIY', 'IIIIIIIIIZ', 'IIIIIIIIXI', 'IIIIIIIIYI', 'IIIIIIIIZI', 'IIIIIIIXII', 'IIIIIIIYII', 'IIIIIIIZII', 'IIIIIIXIII', 'IIIIIIYIII', 'IIIIIIZIII', 'IIIIIXIIII', 'IIIIIYIIII', 'IIIIIZIIII', 'IIIIXIIIII', 'IIIIYIIIII', 'IIIIZIIIII', 'IIIXIIIIII', 'IIIYIIIIII', 'IIIZIIIIII', 'IIXIIIIIII', 'IIYIIIIIII', 'IIZIIIIIII', 'IXIIIIIIII', 'IYIIIIIIII', 'IZIIIIIIII', 'XIIIIIIIII', 'YIIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.39269908+0.j, 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j,
 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j, 0.78539816+0.j,
 1.57079633+0.j, 0.39269908+0.j, 0.

From the qubit operator, we can generate a quantum circuit which models its time evolution.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a handy function do just that:

In [5]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=2),
)
circuit.draw("mpl", style="iqp", scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif" alt="Output of the previous code cell" />

A partir del operador de qubits, podemos generar un circuito cuántico que modele su evolución temporal.
Una vez más, el módulo [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) viene al rescate con una función conveniente para hacer justamente eso:

In [6]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 18 slices.


![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif)

### Paso 2: Optimizar el problema para la ejecución en hardware cuántico
#### Crear segmentos del circuito para retropropagar
Recuerda que la función ``backpropagate`` retropropagará segmentos completos del circuito a la vez, por lo que la elección de cómo segmentar puede tener un impacto en el rendimiento de la retropropagación para un problema dado. Aquí, agruparemos las puertas del mismo tipo en segmentos utilizando la función [slice_by_gate_types](https://docs.quantum.ibm.com/api/qiskit-addon-utils/slicing#slice_by_gate_types).

Para una discusión más detallada sobre la segmentación de circuitos, consulta esta [guía práctica](https://qiskit.github.io/qiskit-addon-utils/how_tos/create_circuit_slices.html) del paquete [qiskit-addon-utils](https://github.com/Qiskit/qiskit-addon-utils).

In [7]:
op_budget = OperatorBudget(max_qwc_groups=8)

#### Backpropagate slices from the circuit

First we specify the observable to be $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, $N$ being the number of qubits. We will backpropagate slices from the time-evolution circuit until the terms in the observable can no longer be combined into eight or fewer qubit-wise commuting Pauli groups.

In [8]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits=num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j,
 0.1+0.j, 0.1+0.j])

#### Restringir cuánto puede crecer el operador durante la retropropagación
Durante la retropropagación, el número de términos en el operador generalmente se acercará rápidamente a $4^N$, donde $N$ es el número de qubits. Cuando dos términos en el operador no conmutan qubit a qubit, necesitamos circuitos separados para obtener los valores esperados correspondientes a ellos. Por ejemplo, si tenemos un observable de 2 qubits $O = 0.1 XX + 0.3 IZ - 0.5 IX$, entonces dado que $[XX,IX] = 0$, una medición en una sola base es suficiente para calcular los valores esperados de estos dos términos. Sin embargo, $IZ$ anticonmuta con los otros dos términos. Por lo tanto, necesitamos una medición de base separada para calcular el valor esperado de $IZ$. En otras palabras, necesitamos dos circuitos, en lugar de uno, para calcular $\langle O \rangle$. A medida que aumenta el número de términos en el operador, existe la posibilidad de que el número requerido de ejecuciones de circuitos también aumente.

El tamaño del operador puede limitarse especificando el argumento ``operator_budget`` de la función ``backpropagate``, que acepta una instancia de [OperatorBudget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-simplify#operatorbudget).

Para controlar la cantidad de recursos adicionales (tiempo) asignados, restringimos el número máximo de grupos de Pauli conmutativos qubit a qubit que el observable retropropagado puede tener. Aquí especificamos que la retropropagación debe detenerse cuando el número de grupos de Pauli conmutativos qubit a qubit en el operador supere 8.

In [9]:
# Backpropagate slices onto the observable
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
# Recombine the slices remaining after backpropagation
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 6 slices.
New observable has 60 terms, which can be combined into 6 groups.
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif" alt="Output of the previous code cell" />

#### Retropropagar segmentos del circuito
Primero especificamos el observable como $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, donde $N$ es el número de qubits. Retropropagaremos segmentos del circuito de evolución temporal hasta que los términos en el observable ya no puedan combinarse en ocho o menos grupos de Pauli conmutativos qubit a qubit.

In [10]:
truncation_error_budget = setup_budget(max_error_per_slice=0.005)

Note that by allocating `5e-3` error per slice for truncation, we are able to remove 1 more slice from the circuit, while remaining within the original budget of eight commuting Pauli groups in the observable. By default, `backpropagate` uses the L1 norm of the truncated coefficients to bound the total error incurred from truncation. For other options refer to the [how-to guide on specifying the p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

In this particular example where we have backpropagated seven slices, the total truncation error should not exceed ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
For further discussion on distributing an error budget across your slices, check out [this how-to guide](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [11]:
# Run the same experiment but truncate observable terms with small coefficients
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", scale=0.6)

Backpropagated 7 slices.
New observable has 82 terms, which can be combined into 8 groups.
After truncation, the error in our observable is bounded by 3.266e-02
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/5e8bae1a-ef18-4eb0-9d2a-1ac7bbdced3b-1.avif" alt="Output of the previous code cell" />

A continuación verás que retropropagamos seis segmentos, y los términos se combinaron en seis grupos, no en ocho. Esto implica que retropropagar un segmento más haría que el número de grupos de Pauli superara ocho. Podemos verificar que este es el caso inspeccionando los metadatos devueltos. Observa también que en esta parte la transformación del circuito es exacta. Es decir, no se truncaron términos del nuevo observable $O'$. El circuito retropropagado y el operador retropropagado producen exactamente el mismo resultado que el circuito y el operador originales.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

In [13]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif)

A continuación, especificaremos el mismo problema con las mismas restricciones sobre el tamaño del observable de salida. Sin embargo, esta vez asignaremos un presupuesto de error a cada segmento utilizando la función [setup_budget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-truncating#setup_budget). Los términos de Pauli con coeficientes pequeños se truncarán de cada segmento hasta que el presupuesto de error se agote, y el presupuesto sobrante se agregará al presupuesto del segmento siguiente. Observa que en este caso, la transformación debida a la retropropagación es aproximada, ya que algunos de los términos en el operador se truncan.

Para habilitar esta truncación, necesitamos configurar nuestro presupuesto de error de la siguiente manera:

In [14]:
pub = (circuit_isa, observable_isa)
bp_pub = (bp_circuit_isa, bp_obs_isa)
bp_trunc_pub = (bp_circuit_trunc_isa, bp_obs_trunc_isa)

Observa que al asignar un error de `5e-3` por segmento para la truncación, podemos eliminar 1 segmento más del circuito, manteniéndonos dentro del presupuesto original de ocho grupos de Pauli conmutativos en el observable. De forma predeterminada, `backpropagate` utiliza la norma L1 de los coeficientes truncados para acotar el error total incurrido por la truncación. Para otras opciones, consulta la [guía práctica sobre la especificación de la norma p](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

En este ejemplo particular donde hemos retropropagado siete segmentos, el error total de truncación no debería exceder ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
Para más información sobre cómo distribuir un presupuesto de error entre sus segmentos, consulta [esta guía práctica](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [15]:
ideal_estimator = Estimator()

# Run the experiments using Estimator primitive to obtain the exact outcome
result_exact = (
    ideal_estimator.run([(circuit, observable)]).result()[0].data.evs.item()
)
print(f"Exact expectation value: {result_exact}")

Exact expectation value: 0.8871244838989416


We shall use <a href="/docs/guides/configure-error-mitigation">resilience_level</a> = 2 for this example.

In [ ]:
options = EstimatorOptions()
options.default_precision = 0.011
options.resilience_level = 2

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run([pub, bp_pub, bp_trunc_pub])

### Step 4: Post-process and return result to desired classical format

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

std_no_bp = job.result()[0].data.stds.item()
std_bp = job.result()[1].data.stds.item()
std_bp_trunc = job.result()[2].data.stds.item()

In [ ]:
print(
    f"Expectation value without backpropagation: {result_no_bp} ± {std_no_bp}"
)
print(f"Backpropagated expectation value: {result_bp} ± {std_bp}")
print(
    f"Backpropagated expectation value with truncation: {result_bp_trunc} ± {std_bp_trunc}"
)

Expectation value without backpropagation: 0.8033194665993642
Backpropagated expectation value: 0.8599808781259016
Backpropagated expectation value with truncation: 0.8868736004169483


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]
stds = [std_no_bp, std_bp, std_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.6, 0.92])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif" alt="Output of the previous code cell" />

## Part B: Scale it up!

Let us now use Operator Backpropagation to study the dynamics of the Hamiltonian of a 50-qubit Heisenberg Spin Chain.

### Step 1: Map classical inputs to a quantum problem

We consider a 50-qubit Hamiltonian $\hat{\mathcal{H}}_{XYZ}$ for the scaled up problem with the same values for the $J$ and $h$ coefficients as in the small-scale example. The observable $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ is also the same as before. This problem is beyond classical brute-force simulation.

In [16]:
num_qubits = 50
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/47cb1ac7-44db-4f96-b49b-e889a920d83c-0.avif" alt="Output of the previous code cell" />

In [17]:
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIII', 'IIIIIIIIIIII

In [18]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIII

### Paso 4: Posprocesar y devolver el resultado al formato clásico deseado

In [19]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=4),
)
circuit.draw("mpl", style="iqp", fold=-1, scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b10d16cf-95da-42c0-9b47-b2e5a8516c82-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution

In [20]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 36 slices.


We specify the `max_error_per_slice` to be 0.005 as before. However, since the number of slices for this large-scale problem is much higher than the small scale problem, allowing an error of 0.005 per slice may end up creating a large overall backpropagation error. We can bound this by specifying `max_error_total` which bounds the total backpropagation error, and we set its value to 0.03 (which is roughly the same as in the small-scale example).

For this large-scale example, we allow a higher value for the number of commuting groups, and set it to 15.

In [21]:
op_budget = OperatorBudget(max_qwc_groups=15)
truncation_error_budget = setup_budget(
    max_error_total=0.03, max_error_per_slice=0.005
)

Let us first obtain the backpropagated circuit and observable without any truncation.

In [22]:
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 7 slices.
New observable has 634 terms, which can be combined into 12 groups.
Note that backpropagating one more slice would result in 1246 terms across 27 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/164e2f00-25b6-4cf1-98f8-8b2886f007ee-1.avif" alt="Output of the previous code cell" />

Now allowing for truncation, we obtain:

In [23]:
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", fold=-1, scale=0.6)

Backpropagated 10 slices.
New observable has 646 terms, which can be combined into 14 groups.
After truncation, the error in our observable is bounded by 2.998e-02
Note that backpropagating one more slice would result in 1226 terms across 29 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/c05a85bc-e5ca-4e02-8c96-98b28811f335-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif)
## Parte B: Escalado a mayor tamaño
Utilicemos ahora la retropropagación de operadores para estudiar la dinámica del hamiltoniano de una cadena de espines de Heisenberg de 50 qubits.
### Paso 1: Mapear las entradas clásicas a un problema cuántico
Consideramos un hamiltoniano de 50 qubits $\hat{\mathcal{H}}_{XYZ}$ para el problema escalado con los mismos valores para los coeficientes $J$ y $h$ que en el ejemplo a pequeña escala. El observable $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ también es el mismo que antes. Este problema está más allá de la simulación clásica por fuerza bruta.

In [24]:
# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile the backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs_trunc.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

In [25]:
print(
    f"2-qubit depth of original circuit: {circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit: {bp_circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit with truncation: {bp_circuit_trunc_isa.depth(lambda x:x.operation.num_qubits==2)}"
)

2-qubit depth of original circuit: 48
2-qubit depth of backpropagated circuit: 40
2-qubit depth of backpropagated circuit with truncation: 36


### Step 3: Execute using Qiskit primitives

In [26]:
pubs = [
    (circuit_isa, observable_isa),
    (bp_circuit_isa, bp_obs_isa),
    (bp_circuit_trunc_isa, bp_obs_trunc_isa),
]

In [27]:
options = EstimatorOptions()
options.default_precision = 0.01
options.resilience_level = 2
options.resilience.zne.noise_factors = [1, 1.2, 1.4]
options.resilience.zne.extrapolator = ["linear"]

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run(pubs)

Para este problema escalado hemos considerado un tiempo de evolución de $0.2$ con $4$ pasos de Trotter. El problema fue seleccionado de modo que esté más allá de la simulación clásica por fuerza bruta, pero pueda ser simulado mediante métodos de redes tensoriales. Esto nos permite verificar el resultado obtenido mediante retropropagación en una computadora cuántica con el resultado ideal.

El valor esperado ideal para este problema, obtenido mediante simulación con redes tensoriales, es $\simeq 0.89$.

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

In [ ]:
print(f"Expectation value without backpropagation: {result_no_bp}")
print(f"Backpropagated expectation value: {result_bp}")
print(f"Backpropagated expectation value with truncation: {result_bp_trunc}")

Expectation value without backpropagation: 0.7887194658035515
Backpropagated expectation value: 0.9532818300978584
Backpropagated expectation value with truncation: 0.8913400398926913


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(0.89)
ax.set_ylim([0.6, 0.98])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/047d448f-aebf-45ff-a81b-83b2d5ca866d-1.avif" alt="Output of the previous code cell" />

Especificamos el `max_error_per_slice` en 0.005 como antes. Sin embargo, dado que el número de segmentos para este problema a gran escala es mucho mayor que para el problema a pequeña escala, permitir un error de 0.005 por segmento podría terminar creando un error total de retropropagación grande. Podemos acotar esto especificando `max_error_total`, que limita el error total de retropropagación, y establecemos su valor en 0.03 (que es aproximadamente el mismo que en el ejemplo a pequeña escala).

Para este ejemplo a gran escala, permitimos un valor más alto para el número de grupos conmutativos, y lo establecemos en 15.